In [2]:
from tqdm import tqdm
from pathlib import Path
import os  # Add this import

import torch

from torch.utils.data import DataLoader

from transformer import Transformer, FrenchEnglishDataset
from tokenizer.bpe_tokenizer import BPETokenizer

/opt/miniconda3/envs/mlab/lib/python3.13/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
device = "cpu"

tokenizer = BPETokenizer()
vocab_size = len(tokenizer.vocab)

model = Transformer(vocab_size=vocab_size,
                    num_layers=2,
                    num_heads=2,
                    ffn_hidden_dim=64,
                    embedding_dim=64,
                    qk_length=64,
                    value_length=64,
                    max_length=5000,
                    dropout=0.1)
model.to(device)

# Load the checkpoint
checkpoint = torch.load('ckpts/model_final_small.pt', map_location=torch.device('cpu'))

# Load the state dictionary into your model
model.load_state_dict(checkpoint)


train_dataset = FrenchEnglishDataset(Path("en-fr-small.csv"), tokenizer=tokenizer, train=True)
train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True, collate_fn=FrenchEnglishDataset.collate_fn)

batch = next(iter(train_loader))
src, tgt = batch

loss_fn = torch.nn.CrossEntropyLoss(label_smoothing=0.1, ignore_index=0)

src = src.to(device)
tgt = tgt.to(device)

src = src.to(torch.int64)
tgt = tgt.to(torch.int64)

model.eval()
with torch.no_grad():
    outputs = model(src, tgt)
    outputs = outputs.view(-1, vocab_size)
    tgt = tgt.view(-1)
    loss = loss_fn(outputs, tgt)
    print(loss)

    # Decode the outputs into English translations
    # Assuming outputs are of shape (batch_size * sequence_length, vocab_size)
    # We need to reshape them back to (batch_size, sequence_length, vocab_size)
    batch_size = src.shape[0]
    sequence_length = tgt.shape[0] // batch_size
    outputs = outputs.view(batch_size, sequence_length, vocab_size)

    # Greedy decoding
    decoded_outputs = []
    for output in outputs:
        # Get the argmax of each token in the sequence
        token_ids = torch.argmax(output, dim=-1)
        
        # Decode the sequence
        decoded_sequence = tokenizer.decode(token_ids)
        
        decoded_outputs.append(decoded_sequence)

    # Print the decoded translations
    for i, translation in enumerate(decoded_outputs):
        print(f"Translation {i + 1}: {translation}")

/var/folders/bv/p40w5hqx32ngltl7lx92x0gc0000gn/T/ipykernel_81681/2253660803.py:18: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  checkpoint = torch.load('ckpts/model_final_s

tensor(5.0797)
Translation 1:  the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the
Translation 2:  the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the
Translation 3:  the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the
Translation 4:  the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the
Translation 5:  the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the
Translation 6:  the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the
